In [1]:
from konlpy.corpus import kobill
from collections import defaultdict

def documentRepresentation5():
    documentList = defaultdict(lambda: defaultdict(int))
    for idx in kobill.fileids():
        for term in kobill.open(idx).read().split():
            documentList[idx][term] += 1
    return documentList

In [2]:
collection = {
    ('Document1', 'This is a a a a a a a a sample'),
    ('Document2', 'This is another sample'),
    ('Document3', 'This is not a sample')
}

In [6]:
DTM = defaultdict(lambda: defaultdict(int))
for docName, docConternt in collection:
    for term in docConternt.lower().split():
        DTM[docName][term] += 1
        
TDM = defaultdict(lambda: defaultdict(int))
for idx, termList in DTM.items():
    for term, freq in termList.items():
        TDM[term][idx] = freq

In [7]:
# N=len(DTM), |V|=len(TDM), DTM.keys(), V=TDM.keys()
# maxTF(in Document1) = max(DTM['Document1'].values())

N = len(DTM)

DTM.values()

dict_values([defaultdict(<class 'int'>, {'this': 1, 'is': 1, 'another': 1, 'sample': 1}), defaultdict(<class 'int'>, {'this': 1, 'is': 1, 'not': 1, 'a': 1, 'sample': 1}), defaultdict(<class 'int'>, {'this': 1, 'is': 1, 'a': 8, 'sample': 1})])

In [19]:
from math import log2

TWM = defaultdict(lambda:defaultdict(float))
for idx, termList in DTM.items():
    maxTF = max(termList.values())
    for term, freq in termList.items():
        TF = freq/maxTF
        IDF = log2(N/len(TDM[term]))
        TWM[term][idx] = TF*IDF
        
        

In [20]:
TWM # 다양한 빈도속에서 옥석이 가려낼 것임

defaultdict(<function __main__.<lambda>()>,
            {'this': defaultdict(float,
                         {'Document2': 0.0,
                          'Document3': 0.0,
                          'Document1': 0.0}),
             'is': defaultdict(float,
                         {'Document2': 0.0,
                          'Document3': 0.0,
                          'Document1': 0.0}),
             'another': defaultdict(float, {'Document2': 1.584962500721156}),
             'sample': defaultdict(float,
                         {'Document2': 0.0,
                          'Document3': 0.0,
                          'Document1': 0.0}),
             'not': defaultdict(float, {'Document3': 1.584962500721156}),
             'a': defaultdict(float,
                         {'Document3': 0.5849625007211562,
                          'Document1': 0.5849625007211562})})

In [21]:
from konlpy.corpus import kobill
from konlpy.tag import Komoran
from math import sqrt

ma = Komoran()
DTM = defaultdict(lambda: defaultdict(int))
for idx in kobill.fileids():
#     ma.pos() : 형태소/품사
#     ma.morphs() : 형태소, 형태소 ...
#     ma.nouns() : 명사, 명사 ...
    for term in kobill.open(idx).read().split():
        for token in ma.pos(term):
            DTM[idx]["/".join(token)] += 1
#             DTM[docName][term] += 1
        
TDM = defaultdict(lambda: defaultdict(int))
for idx, termList in DTM.items():
    for term, freq in termList.items():
        TDM[term][idx] = freq
        
TWM = defaultdict(lambda:defaultdict(float))
DVL = defaultdict(float)

N = len(DTM)
for idx, termList in DTM.items():
    maxTF = max(termList.values())
    for term, freq in termList.items():
        TF = freq/(maxTF + 1)
        IDF = log2(N/len(TDM[term]))
        TWM[term][idx] = TF*IDF
        DVL[idx] += TWM[term][idx]**2
        
for idx, length in DVL.items():
    DVL[idx] = sqrt(length)

In [23]:
DVL

defaultdict(float,
            {'1809890.txt': 1.5139677253857564,
             '1809891.txt': 1.51273592886795,
             '1809892.txt': 1.389903004021774,
             '1809893.txt': 1.5948700159365905,
             '1809894.txt': 3.4211226711134883,
             '1809895.txt': 2.5130955975730007,
             '1809896.txt': 2.0256978581464455,
             '1809897.txt': 2.9143263234869248,
             '1809898.txt': 2.4906165500672466,
             '1809899.txt': 2.059529616511176})

# similarity
## 유클리디안 디스턴스는 점으로 보기 때문에 문서의 의미를 전부 못담고 패널티가 너무크다
## -> 그래서 세타값인 코사인 유사도를 가지고 할 것이다.
## 단위벡터를 사용하기 때문에

In [31]:
from nltk.tokenize import sent_tokenize

query = '국방의 의무와 보편적 의무에 대한 의무를 찾아주세요.'

TQM = defaultdict(int)
QWM = defaultdict(float)

for term in query.split():
    for token in ma.pos(term):
        TQM['/'.join(token)] += 1
        
alpha = 0.5
maxTF = max(TQM.values())
for term, ferq in TQM.items():
    TF = alpha + (1-alpha)*(freq/maxTF)
    DF = len(TWM[term]) if len(TWM[term]) > 0 else 1
    IDF = log2(N/DF)
    QWM[term] = TF*IDF
    
candidateList = defaultdict(float)
for term, weight1 in QWM.items():
    for doc, weight2 in TWM[term].items():
        innerProduct = weight1 * weight2
        candidateList[doc] += innerProduct
        
for doc, sim in candidateList.items():
    candidateList[doc] = sim/DVL[doc]
    
K = 5
for doc, sim in sorted(candidateList.items(), key=lambda x:x[1], reverse=True)[:K]:
    print('문서이름:{0} / 유사도:{1:.4f}'.format(doc, sim))
    print(sent_tokenize(kobill.open(doc).read())[:3])

문서이름:1809899.txt / 유사도:0.0509
['결혼중개업의 관리에 관한 법률 일부개정법률안\n\n(한선교의원 대표발의 )\n\n 의 안\n 번 호\n\n9899\n\n발의연월일 : 2010.', '11.', '15.']
문서이름:1809898.txt / 유사도:0.0215
['국군부대의 소말리아 해역 파견연장 동의안\n\n의안\n                                                                  제출연월일 :  2010.', '11.', '15.']
문서이름:1809897.txt / 유사도:0.0204
['국군부대의 아랍에미리트(UAE)군 교육훈련 지원 등에 \n관한 파견 동의안\n\n의안\n                                                                  제출연월일 :  2010.', '11.', '15.']
문서이름:1809891.txt / 유사도:0.0040
['국가공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9891\n\n발의연월일 : 2010.', '11.', '12.']
문서이름:1809890.txt / 유사도:0.0040
['지방공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9890\n\n발의연월일 : 2010.', '11.', '12.']


In [43]:
query = '국방의 의무와 보편적 의무에 대한 의무를 찾아주세요.'

TQM = defaultdict(int)
QWM = defaultdict(float)

for term in query.split():
    for token in ma.pos(term):
        TQM['/'.join(token)] += 1
TQM

defaultdict(int,
            {'국방/NNP': 1,
             '의/JKG': 1,
             '의무/NNG': 3,
             '와/JC': 1,
             '보편/NNG': 1,
             '적/XSN': 1,
             '에/JKB': 1,
             '대하/VV': 1,
             'ㄴ/ETM': 1,
             '를/JKO': 1,
             '찾/VV': 1,
             '아/EC': 1,
             '주/VX': 1,
             '시/EP': 1,
             '어요/EF': 1,
             './SF': 1})

In [50]:
alpha = 0.5
maxTF = max(TQM.values())
for term, ferq in TQM.items():
    TF = alpha + (1-alpha)*(freq/maxTF)
    DF = len(TWM[term]) if len(TWM[term]) > 0 else 1
    IDF = log2(N/DF)
    QWM[term] = TF*IDF

In [51]:
N, QWM

(10,
 defaultdict(float,
             {'국방/NNP': 1.5479520632582413,
              '의/JKG': 0.0,
              '의무/NNG': 2.214618729924908,
              '와/JC': 0.10133539563003337,
              '보편/NNG': 2.214618729924908,
              '적/XSN': 0.10133539563003337,
              '에/JKB': 0.0,
              '대하/VV': 0.0,
              'ㄴ/ETM': 0.0,
              '를/JKO': 0.0,
              '찾/VV': 2.214618729924908,
              '아/EC': 0.0,
              '주/VX': 0.34304878188650545,
              '시/EP': 2.214618729924908,
              '어요/EF': 2.214618729924908,
              './SF': 0.0}))

In [52]:
candidateList = defaultdict(float)
for term, weight1 in QWM.items():
    for doc, weight2 in TWM[term].items():
        innerProduct = weight1 * weight2
        candidateList[doc] += innerProduct
        
for doc, sim in candidateList.items():
    candidateList[doc] = sim/DVL[doc]

In [53]:
from nltk.tokenize import sent_tokenize

K = 5 
for doc, sim in sorted(candidateList.items(), key=lambda x:x[1], reverse=True)[:K]:
    print("문서이름 : {0} / 유사도:{1:.4f}".format(doc, sim))
    print(sent_tokenize(kobill.open(doc).read())[:3])
    print("")

문서이름 : 1809899.txt / 유사도:0.0509
['결혼중개업의 관리에 관한 법률 일부개정법률안\n\n(한선교의원 대표발의 )\n\n 의 안\n 번 호\n\n9899\n\n발의연월일 : 2010.', '11.', '15.']

문서이름 : 1809898.txt / 유사도:0.0215
['국군부대의 소말리아 해역 파견연장 동의안\n\n의안\n                                                                  제출연월일 :  2010.', '11.', '15.']

문서이름 : 1809897.txt / 유사도:0.0204
['국군부대의 아랍에미리트(UAE)군 교육훈련 지원 등에 \n관한 파견 동의안\n\n의안\n                                                                  제출연월일 :  2010.', '11.', '15.']

문서이름 : 1809891.txt / 유사도:0.0040
['국가공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9891\n\n발의연월일 : 2010.', '11.', '12.']

문서이름 : 1809890.txt / 유사도:0.0040
['지방공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9890\n\n발의연월일 : 2010.', '11.', '12.']



In [72]:
# kobill.open("1809899.txt").read()

In [54]:
TWM["의무/NNG"]

defaultdict(float, {'1809899.txt': 0.04678771964630088})

### 유클리디안으로 하기

In [62]:
candidateList = defaultdict(float)
for term, docList in TWM.items():
    for doc, weight1 in docList.items():
        weight2 = QWM[term]
        candidateList[doc] += (weight1 - weight2)**2 
        
for doc, sim in candidateList.items():
    candidateList[doc] = sqrt(sim)

for doc in DTM:
    print(doc, len(kobill.open(doc).read().split()), len(DTM[doc]), sum(DTM[doc].values()))
candidateList

1809890.txt 841 484 1947
1809891.txt 834 480 1941
1809892.txt 984 516 2250
1809893.txt 840 490 1942
1809894.txt 242 206 520
1809895.txt 394 272 889
1809896.txt 1939 624 4217
1809897.txt 788 409 1430
1809898.txt 821 405 1466
1809899.txt 1677 532 3899


defaultdict(float,
            {'1809890.txt': 1.5550626224016812,
             '1809891.txt': 1.5538634034460859,
             '1809892.txt': 1.435498611350987,
             '1809893.txt': 1.633815708477598,
             '1809897.txt': 3.302848274077221,
             '1809898.txt': 2.9377263509745783,
             '1809894.txt': 3.4211226711134897,
             '1809895.txt': 2.5157501870556507,
             '1809896.txt': 2.0572101455799565,
             '1809899.txt': 2.992807843973513})

In [63]:
from nltk.tokenize import sent_tokenize

K = 5
for doc, sim in sorted(candidateList.items(), key=lambda x:x[1])[:K]:
    print('문서이름:{0} / 거리:{1:.4f}'.format(doc, sim))
    print(sent_tokenize(kobill.open(doc).read())[:3])
    print()

문서이름:1809892.txt / 거리:1.4355
['교육공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9892\n\n발의연월일 : 2010.', '11.', '12.']

문서이름:1809891.txt / 거리:1.5539
['국가공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9891\n\n발의연월일 : 2010.', '11.', '12.']

문서이름:1809890.txt / 거리:1.5551
['지방공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9890\n\n발의연월일 : 2010.', '11.', '12.']

문서이름:1809893.txt / 거리:1.6338
['남녀고용평등과 일 ·가정 양립 지원에 관한 법률 \n\n일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9893\n\n발의연월일 : 2010.', '11.', '12.']

문서이름:1809896.txt / 거리:2.0572
['행정절차법 일부개정법률안\n\n(유선호의원 대표발의 )\n\n 의 안\n 번 호\n\n9896\n\n발의연월일 : 2010.', '11.', '15.']

